[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/owenip/LNP-MOD/blob/cli/colab_notebooks/LNP-MOD.ipynb)


## Setup environment

### Make sure the notebook is using GPU via `nvidia-smi` command.
In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

*This tool can also work without GPU but the processing time will be significantly longer.*

In [ ]:
#@title ##Run this cell to check if you have GPU access { display-mode: "form" }
from IPython import display
%pip install tensorflow

display.clear_output()

import tensorflow as tf

if tf.test.gpu_device_name() != '/device:GPU:0':
    print('\nYou do not have GPU access.')
    print('\nDid you change your runtime ?')
    print('\nIf the runtime setting is correct then Google did not allocate a GPU for your session')
    print('\nExpect slow performance. To access GPU try reconnecting later')

else:
    print('You have GPU access')
    !nvidia-smi

# Install SAM Framework

In [ ]:
from google.colab import userdata

!pip install -q git+https://github.com/facebookresearch/segment-anything.git

github_token = userdata.get('github_token')
!git clone https://{github_token}@github.com/owenip/LNP-MOD.git
!cd /content/LNP-MOD && git checkout cli && git pull
!pip install -e /content/LNP-MOD/

### Install required library

In [ ]:
# Install ultralytics library
from IPython import display
%pip install -U ultralytics shapely 

display.clear_output()
import ultralytics
ultralytics.checks()

### **Mount your Google Drive**

In [ ]:
#@title ##Play the cell to connect your Google Drive to Colab { display-mode: "form" }

#@markdown 1. Click 'Connect to Google Drive' at the pop up window

#@markdown 2. Sign in your Google Account.

#@markdown 3. Click 'Allow' to give this notebook access to the data on the drive

#@markdown * Once this is done, your data are available in the Files tab on the top left of notebook.

# Mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/drive')

# Force session restart
exit(0)

### Setup Dataset and Model

In [ ]:
# @title ### Setup Dataset and Model { display-mode: "form" }
# @markdown ### Provide the path to the dataset folder

import os
import glob

input_images_folder = "/content/drive/Shareddrives/Bleb_Counting/raw_data/" # @param {type:"string"}
path_to_od_model = "/content/objection_detection.pt"
path_to_seg_model = "/content/segmentation_model.pt"

def check_file_exist(file_path):
    if os.path.exists(file_path) == False:
        raise Exception('File does not exist: ' + file_path)

def get_supported_images_path_list(images_folder):
    images = []
    # Exclude '*.jpg', '*.jpeg' as this format are always used for the overview image
    supported_images = ('*.tif', '*.tiff', '*.png', '*.bmp',
                        '*.dng', '*.webp', '*.pfm', '*.mpo')
    for image_type in supported_images:
        images.extend(glob.glob(images_folder + image_type))

    return sorted(images)

input_images_folder = os.path.join(input_images_folder, '')
check_file_exist(input_images_folder)
check_file_exist(path_to_od_model)
check_file_exist(path_to_seg_model)

num_of_supported_images = len(get_supported_images_path_list(input_images_folder))
if num_of_supported_images == 0:
    raise Exception('No supported images found in the dataset folder')
else:
    print('Number of supported images found in the dataset folder: ' + str(num_of_supported_images))

# @markdown #*Play the cell to ensure the dataset folder contains supported images*

In [ ]:
# @title ### Setup Dataset and Model { display-mode: "form" }
# @markdown ### Provide the path to the dataset folder

import os
import glob

input_images_folder = "/content/imgs_dir" # @param {type:"string"}
input_images_folder = os.path.join(input_images_folder, '')

# @markdown ### Provide the path to the objection detection model
# path_to_od_model = "/content/objection_detection.pt"
# path_to_seg_model = "/content/segmentation_model.pt"

!python /content/LNP-MOD/lnp_mod/inference.py --input_dir "{input_images_folder}"

# @markdown #*Play the cell to run LNP-MOD*